In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Olist'in Tahmini Teslimat Süresini (ETA) İyileştirme

👉 `Estimated Time of Arrival (ETA)`, modern e-ticaret hizmetlerinin temel bileşenlerinden biridir. Şirketlerin müşteriye teslimat için tahmini bir tarih göstererek beklentileri yönetmesini sağlar.

🎯 Bu çalışmada, bir siparişin `bekleme süresini tahmin eden` bir model kuracağız.

☑️ Olist'in kendi tahminleri `expected_wait_time` değişkeninde zaten mevcut.

🚀 Hadi daha iyi tahminler yapmaya çalışalım ve bu özetin sonunda şunları karşılaştıralım:
* bizim `estimated_wait_time`
* Olist'in `expected_wait_time`
* teslim edilen her sipariş için gerçek `wait_time`

## `orders` veri setini yükle ve kontrol et

👉 `Order().get_training_data()` (`distance_seller_customer` dahil)

In [ ]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)

In [ ]:
orders.head(3)

In [ ]:
orders.info()

## Temel Model - Olist'in mevcut ETA modeli

🐣  Optimize etmek istediğimiz temel seviye (baseline) nedir ❓

👉 Olist'in `expected_wait_time` değeri ile gerçek `wait_time` arasındaki fark

📈 İlgili histogramlarını çiz

In [ ]:
# YOUR CODE HERE

👉 Medyan bekleme süresi nedir? Medyan beklenen bekleme süresi nedir?

In [ ]:
# YOUR CODE HERE

👉 Temel RMSE'yi hesapla

ℹ️ Burada ele alınan hatalar, Olist'in bekleme süresi tahminindeki sapmalarıdır.

In [ ]:
# YOUR CODE HERE

## OLS çok değişkenli model

👉 Bu sefer `statsmodel.api.OLS` yöntemini kullanarak bir OLS modeli oluşturacağız.

(`statsmodel.formula.api.ols` yerine)

### X özellikleri ve Y hedefi ile veri setini hazırla

In [ ]:
# YOUR CODE HERE

⚖️ OLS sonucunda elde edilecek regresyon katsayılarını karşılaştırabilmek için X'in her bir özelliğini standartlaştır (`z-score`):

In [ ]:
# YOUR CODE HERE

In [ ]:
# Check how different is it from the "formula" in statsmodel
import patsy
patsy.dmatrix('price + distance_seller_customer', orders)

⚠️ Bu `statsmodel.api.OLS` ile, özellikler veri setine her zaman sabit bir "intercept" (kesme) eklememiz gerekir:

In [ ]:
# YOUR CODE HERE

### `statsmodels` ile OLS'i çalıştır ve `summary` çıktısını yazdır

In [ ]:
# YOUR CODE HERE

## OLS modelinin performansını kontrol et

### Uyum iyiliği (goodness of fit)

👉 En iyi tahminleri kim yapıyor: sen mi, Olist mi?

- Tahminlerini hesapla ve `y_pred` değişkeninde sakla

- Şunları da saklayabilirsin:
    - Olist'in tahminlerini `y_expected_olist` değişkeninde
    - gerçek bekleme sürelerini `y_true` değişkeninde


In [ ]:
# YOUR CODE HERE

📈 `y_pred`, `y_expected_olist` ve `y_true` değerlerini aynı grafikte çiz.

In [ ]:
# YOUR CODE HERE

👉 Artık hatalar *senin* hataların olacak şekilde RMSE'yi hesapla!

In [ ]:
# YOUR CODE HERE

Hatırlatma olarak, Olist'in RMSE'si şuna eşitti: 👇

In [ ]:
baseline_RMSE

😏 Gerçekten Olist'in Veri Bilimcilerini yendik mi? Pek de öyle diyemeyiz!

- Birçok e-ticaret ve teslimat servisinde ETA çoğu zaman yüksek tahmin edilir...
- Müşteriler siparişlerini beklenenden erken aldıklarında memnun olma olasılıkları daha yüksektir!

### Kısmi korelasyonlar

👉 `sm.graphics.plot_partregress_grid` ile kısmi korelasyonları görselleştir

In [ ]:
# YOUR CODE HERE

**Bu grafikleri nasıl okuruz?**

👉 Örneğin sol alt köşedeki grafiğe odaklanalım:

<u>`wait_time` ile `distance` arasındaki kısmi regresyon</u>

- Her nokta veri setimizdeki bir sipariştir


- `y-ekseni`: `distance` hariç tüm özellikleri kullanarak elde edilen `wait_time` artık (residual) değerleri

    - Bu artıklar, `distance` olmadan açıklanamayan `wait_time` hakkındaki kalan bilgiyi içerir


- `x-ekseni`: diğer tüm özellikler (price, freight_value, vb...) kullanılarak `distance` tahmininin artık (residual) değerleri
    - Bu artıklar, modeldeki diğer özelliklerle zaten açıklanmayan ve `distance` değişkeninin getirdiği yeni bilgiyi içerir.


### Çıkarımsal koşullar

⚠️ Artık (residual) değerlerini kontrol et. Normal dağılıma uyuyorlar mı?

In [ ]:
# YOUR CODE HERE

🎁 Bir dağılımın Gauss (normal) olup olmadığını daha hassas görselleştirmenin bir yolu <a href="https://www.statsmodels.org/stable/generated/statsmodels.graphics.gofplots.qqplot.html">**`QQplot`**</a>'tur.

📈 Aşağıya artık (residual) değerlerinin QQplot'unu çiz.

💡 QQ plot ile az önce çizilen yoğunluk (density) grafiği arasındaki ilişkiyi mutlaka anla..

📚 <a href="https://stats.stackexchange.com/questions/101274/how-to-interpret-a-qq-plot">Stats.StackExchange.com - QQplot yorumlama</a>

- `x-axis` : bir Gauss dağılımının teorik kantilleri
- `y-axis`: sıralanmış değişkenin kantilleri

💡 Eğer <span style="color:blue">mavi</span> noktalar, özdeşlik (identity) fonksiyonu olan <span style="color:red">kırmızı çizgi</span> üzerinde hizalanıyorsa, değişken Gauss dağılımına sahiptir!

In [ ]:
sm.qqplot(residuals_lewagon, line='s');

#### Bazı QQplot örnekleri:

QQ plot'ların nasıl çizildiğini anlamak için aşağıdaki hücreleri çalıştır:

In [ ]:
from scipy import stats
stats.norm.rvs(0,1)

In [ ]:
# lets check QQ plots with our own distributions
from scipy import stats

norm_1 = [stats.norm.rvs(0,1) for x in range(1000)]
norm_2 = [stats.norm.rvs(5,1) for x in range(1000)]

sns.kdeplot(norm_1)
sns.kdeplot(norm_2)
plt.xlim(right=10)

In [ ]:
sm.qqplot(np.asarray(norm_1), line='s');

In [ ]:
bimodal = norm_1 + norm_2
sns.kdeplot(bimodal)
plt.xlim(right=10)

In [ ]:
sm.qqplot(np.asarray(bimodal), line='s');

<img src="https://wagon-public-datasets.s3.amazonaws.com/04-Decision-Science/03-Linear-Regression/qq_plot_interpretation.png" width=400>

#### Rastgele olmayan bir desen yakalayıp yakalayamayacağımızı görmek için artıklar (residuals) vs. uyumlanan (fitted) değerleri kontrol et

In [ ]:
sns.scatterplot(x = y_pred, y = residuals_lewagon, alpha=0.01)
plt.xlim(xmin=0, xmax=50)

## Daha Fazla Özellik? (Opsiyonel)

Burada ürün detaylarından (boyut, kategori vb.) ek özellikler ekleyebiliriz.

### Her siparişle ilişkili ürün detaylarını ekle (ürün boyutları toplamı, ağırlıklar toplamı vb.)

In [ ]:
from olist.data import Olist
data = Olist().get_data()
products = data['products']
order_items = data['order_items']

In [ ]:
order_items_products = (
    order_items.groupby(["order_id", "product_id"], as_index=False)
    .agg({"order_item_id": "count"})
    .rename(columns={"order_item_id": "order_item_count"})
    .merge(products.drop(["product_category_name"], axis=1), on="product_id")
)
order_items_products

In [ ]:
# Since we want the sum of product weights per order from our new DataFrame, we need to multiply weight by product count
order_items_products['product_weight_g'] = order_items_products['product_weight_g'] *  order_items_products['order_item_count']

order_items_features = order_items_products.groupby("order_id", as_index=False).agg(
    {
        "product_weight_g": "sum",
        "product_length_cm": "max",
        "product_height_cm": "max",
        "product_width_cm": "max",
    }).rename(columns={
        "product_weight_g": "product_weight_g_sum",
        "product_length_cm": "product_length_cm_max",
        "product_height_cm": "product_height_cm_max",
        "product_width_cm": "product_width_cm_max",
    })
order_items_features

In [ ]:
orders_with_details = orders.merge(order_items_features, on="order_id", how="left").dropna()

In [ ]:
X2 = orders_with_details[[
        "number_of_items",
        "number_of_sellers",
        "price",
        "freight_value",
        "distance_seller_customer",
        "product_weight_g_sum",
        "product_length_cm_max",
        "product_height_cm_max",
        "product_width_cm_max"]].copy()
y2 = orders_with_details["wait_time"].copy()

In [ ]:
# standardize (z-normalize) each features of X to compare coefficients
for f in X2.columns:
    mu = X2[f].mean()
    sigma = X2[f].std()
    X2[f] = X2[f].map(lambda x: (x - mu) / sigma)

In [ ]:
X2['constant'] = 1

In [ ]:
model2 = sm.OLS(y2, X2).fit()

In [ ]:
model2.summary()